# 2014

Ahora vamos con los de 2014. Algo que se considera desde el ENADID es. 

¿Por qué no usar códigos anteriores?

La respuesta es que los formatos son muy diferentes. Sin embargo, una imagen vale más que 1000 palabras. Entonces, vamos a ver si podemos usar enut19 para calcular enut14

In [19]:
import pandas as pd

enut14 = pd.read_excel('data\\fd_enut14.xls', sheet_name=None)
lsheets = list(enut14.keys())
print(lsheets)

['TVivienda', 'THogar', 'TSDem', 'TModulo1', 'TModulo2', 'TModulo3', 'TNoResidentes']


Serán modificaciones leves, pero se resume a repetir el proceso, pero modificando ciertos valores. Entonces vamos de nuevo.

In [20]:
test_sh = enut14[lsheets[0]].replace({'Mnemónico' : 'Nemónico'})
n_cols = test_sh.shape[1]
new_cols = [f'C{i}' for i in range(n_cols)]
test_sh.columns = new_cols
test_sh.iloc[:,0] = test_sh.isna().sum(axis = 1)
dfn_1 = test_sh[test_sh.iloc[:, 0] == n_cols - 1]
dfn_1.head()

,C0,C1,C2,C3,C4,C5,C6,C7,C8
1,8.0,ESTRUCTURA DEL ARCHIVO,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8.0,Encuesta Nacional sobre Uso del Tiempo 2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,8.0,NaN,Especifica el rango de opciones que aceptan el...,NaN,NaN,NaN,NaN,NaN,NaN
13,8.0,LLAVE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,8.0,Control de vivienda,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Observando, se parece mucho al usado en el ENADID. Pero ese no esta generalizado, por lo que será complicado reutilizarlo, pero se puede tomar como referencia para la implementación a realizar

In [21]:
table_index = list(dfn_1.index)
table_names = list(dfn_1.loc[:,'C1'])
case_one = ['Pregunta', '(1)']
compare_list = [[tn] + case_one for tn in table_names]
sheet_list = [test_sh.loc[ti: ti + 2,'C1'] for ti in table_index]
ithree = []
i = 0
for ish in sheet_list:
    u = len(ish)
    v = len(case_one) + 1
    if u == v:
        ithree.append(i)
    i += 1
com_three = [compare_list[i] for i in ithree]
sht_three = [sheet_list[i] for i in ithree]

equals = [(com_three[i] == sht_three[i]).sum() for i in range(len(sht_three))]
equals_one = []
for i in range(len(equals)):
        if equals[i] == 3:
            equals_one.append(table_index[i])

case_two = ['nan', 'Pregunta', '(1)']
compare_list = [[tn] + case_two for tn in table_names]
sheet_list = [test_sh.loc[ti: ti + 3,'C1'].astype(str) for ti in table_index]
ifour = []
i = 0
for ish in sheet_list:
    u = len(ish)
    v = len(case_two) + 1
    if u == v:
        ifour.append(i)
    i += 1
com_four = [compare_list[i] for i in ifour]
sht_four = [sheet_list[i] for i in ifour]
equals = [(com_four[i] == sht_four[i]).sum() for i in range(len(sht_four))]
for i in range(len(equals)):
        if equals[i] == 4:
            equals_one.append(table_index[i])
equals_one.sort()
dflt = [test_sh.loc[equals_one[i]: equals_one[i + 1] - 1 ,:] for i in range(len(equals_one) - 1)]
dflt.append(test_sh.loc[equals_one[-1]: ,:])
sheet_df = pd.DataFrame()


In [22]:
subdf = pd.DataFrame()

for v in dflt:
    v = v.iloc[:, 1:].dropna(axis = 1, how = 'all').dropna(axis = 0, how = 'all').astype(str)
    name = v.iloc[0, 0]
    cols = list(v.iloc[1,:])
    v.columns = cols
    try:
        j = cols.index('nan')
        for i in range(v.shape[0]):
            if v.iloc[i,j] != 'nan':
                v.iloc[i, 0] = v.iloc[i, 1]
        v = v.drop(columns='nan')
    except:
        print('HI')
    v = v.iloc[3:, :].replace({'nan': pd.NA}).ffill().iloc[1:,:]
    v['SubNombre'] = name
    subdf = pd.concat([subdf, v])
subdf

HI
HI


,Pregunta,Nemónico,Tipo,Códigos Válidos,Concepto,Long.,SubNombre
17,Control de vivienda,CONTROL,Carácter,0100001…3299999,<NA>,7,LLAVE
18,Control de vivienda,VIV_SEL,Carácter,01…99,<NA>,2,LLAVE
28,1.1 ¿De qué material es la mayor parte del pis...,P1_1,Carácter,1,Tierra,1,SECCIÓN I. CARACTERÍSTICAS DE LA VIVIENDA
29,1.1 ¿De qué material es la mayor parte del pis...,P1_1,Carácter,2,Cemento o firme,1,SECCIÓN I. CARACTERÍSTICAS DE LA VIVIENDA
30,1.1 ¿De qué material es la mayor parte del pis...,P1_1,Carácter,3,"Madera, mosaico u otro material",1,SECCIÓN I. CARACTERÍSTICAS DE LA VIVIENDA
...,...,...,...,...,...,...,...
154,Unidad primaria de muestreo,UPM_DIS,Carácter,01001…32999,Localidades con menos de 2 500 habitantes,5,Campos empleados para el diseño muestral
155,Estrato socioeconómico**,UPM_DIS,Carácter,01001…32999,Localidades con menos de 2 500 habitantes,5,Campos empleados para el diseño muestral
156,Unidad primaria de muestreo,UPM_DIS,Carácter,01001…32999,Localidades con menos de 2 500 habitantes,5,Campos empleados para el diseño muestral
158,* No aplica para la base de población indígena.,UPM_DIS,Carácter,01001…32999,Localidades con menos de 2 500 habitantes,5,Campos empleados para el diseño muestral


In [23]:
enut14_all = pd.DataFrame()
for sh in lsheets:
    test_sh = enut14[sh]
    n_cols = test_sh.shape[1]
    new_cols = [f'C{i}' for i in range(n_cols)]
    test_sh.columns = new_cols
    test_sh.iloc[:,0] = test_sh.isna().sum(axis = 1)
    dfn_1 = test_sh[test_sh.iloc[:, 0] == n_cols - 1]
    table_index = list(dfn_1.index)
    table_names = list(dfn_1.loc[:,'C1'])
    case_one = ['Pregunta', '(1)']
    compare_list = [[tn] + case_one for tn in table_names]
    sheet_list = [test_sh.loc[ti: ti + 2,'C1'] for ti in table_index]
    ithree = []
    i = 0
    for ish in sheet_list:
        u = len(ish)
        v = len(case_one) + 1
        if u == v:
            ithree.append(i)
        i += 1
    com_three = [compare_list[i] for i in ithree]
    sht_three = [sheet_list[i] for i in ithree]

    equals = [(com_three[i] == sht_three[i]).sum() for i in range(len(sht_three))]
    equals_one = []
    for i in range(len(equals)):
            if equals[i] == 3:
                equals_one.append(table_index[i])

    case_two = ['nan', 'Pregunta', '(1)']
    compare_list = [[tn] + case_two for tn in table_names]
    sheet_list = [test_sh.loc[ti: ti + 3,'C1'].astype(str) for ti in table_index]
    ifour = []
    i = 0
    for ish in sheet_list:
        u = len(ish)
        v = len(case_two) + 1
        if u == v:
            ifour.append(i)
        i += 1
    com_four = [compare_list[i] for i in ifour]
    sht_four = [sheet_list[i] for i in ifour]
    equals = [(com_four[i] == sht_four[i]).sum() for i in range(len(sht_four))]
    for i in range(len(equals)):
            if equals[i] == 4:
                equals_one.append(table_index[i])
    equals_one.sort()
    dflt = [test_sh.loc[equals_one[i]: equals_one[i + 1] - 1 ,:] for i in range(len(equals_one) - 1)]
    dflt.append(test_sh.loc[equals_one[-1]: ,:])
    for v in dflt:
        v = v.iloc[:, 1:].dropna(axis = 1, how = 'all').dropna(axis = 0, how = 'all').astype(str)
        name = v.iloc[0, 0]
        cols = list(v.iloc[1,:])
        v.columns = cols
        try:
            j = cols.index('nan')
            for i in range(v.shape[0]):
                if v.iloc[i,j] != 'nan':
                    v.iloc[i, 0] = v.iloc[i, 1]
            v = v.drop(columns='nan')
        except:
            print('HI')
        v = v.iloc[3:, :].replace({'nan': pd.NA}).ffill().iloc[1:,:]
        v['SubNombre'] = name
        subdf = pd.concat([subdf, v])
    enut14_all = pd.concat([enut14_all, subdf])   


HI
HI
HI
HI
HI
HI
HI
HI
HI
HI
HI


In [24]:
enut14_all.to_excel('results\\enut14_all.xlsx')